In [1]:
import logging
import time
import numpy as np

import torch
import torchvision
import torchvision.transforms as transforms
from torch import nn, optim
import torch.nn.functional
from torch.utils.data import DataLoader, Dataset, Subset
from torchvision import datasets
from torchvision.transforms import ToTensor, Normalize, Compose, Lambda
from torch.nn.functional import normalize

In [2]:
import sys
import logging.config

logging.config.dictConfig({
    "version": 1,
    "disable_existing_loggers": True,
    "formatters": {
        "default": {
            "format": "{levelname} {asctime} | {message}",
            "style": "{",
        },
    },
    "handlers": {
        "console": {
            "class": "logging.StreamHandler",
            "formatter": "default",
            "stream": sys.stdout,
        }
    },
    "root": {
        "level": "DEBUG",
        "handlers": ["console"],
    },
})


# Get root logger (all other loggers will be derived from this logger's
# properties)
logger = logging.getLogger()
logger.warning("I will output to terminal")  # No output in notebook, goes to terminal

# assuming only a single handler has been setup (seems 
# to be default in notebook), set that handler to go to stdout.
# logger.handlers[0].stream = sys.stdout

logger.warning("FOO")  # Prints: WARNING:root:FOO
logger.info("hello")

# Other loggers derive from the root logger, so you can also do:
logger2 = logging.getLogger("logger2")
logger2.warning("BAR")  # Prints: WARNING:logger2:BAR

WARNING 2023-10-24 15:24:25,110 | I will output to terminal
WARNING 2023-10-24 15:24:25,113 | FOO
INFO 2023-10-24 15:24:25,115 | hello
WARNING 2023-10-24 15:24:25,117 | BAR


In [3]:
# Set hardware

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)

In [4]:
def GlobalContrastNormalization(tensor: torch.tensor, scale='l2'):
    assert scale in ('l1', 'l2')
    n_features = int(np.prod(tensor.shape))
            
    tensor = tensor - torch.mean(tensor)
    
    if (scale == 'l1'):
        tensor = tensor / torch.mean(torch.abs(tensor))
    
    if (scale == 'l2'):
        tensor = tensor / torch.sqrt(torch.sum(tensor ** 2) / n_features)
    
    return tensor

In [5]:
def get_target_label_idx(labels, targets):
    """
    Get the indices of labels that are included in targets.
    :param labels: array of labels
    :param targets: list/tuple of target labels
    :return: list with indices of target labels
    """
    return [idx for idx, label in enumerate(labels) if label in targets]

In [6]:
normal_class = 0

n_classes = 2
normal_classes = tuple([normal_class])
outlier_classes = list(range(0, 10))
outlier_classes.remove(normal_class)

min_max = [(-0.8826567065619495, 9.001545489292527),
           (-0.6661464580883915, 20.108062262467364),
           (-0.7820454743183202, 11.665100841080346),
           (-0.7645772083211267, 12.895051191467457),
           (-0.7253923114302238, 12.683235701611533),
           (-0.7698501867861425, 13.103278415430502),
           (-0.778418217980696, 10.457837397569108),
           (-0.7129780970522351, 12.057777597673047),
           (-0.8280402650205075, 10.581538445782988),
           (-0.7369959242164307, 10.697039838804978)]

transform = Compose([ToTensor(), 
                     Lambda(lambda x: GlobalContrastNormalization(x, scale='l1')), 
                     Normalize([min_max[normal_class][0]], 
                               [min_max[normal_class][1] - min_max[normal_class][0]])])
target_transform = Lambda(lambda x: int(x in outlier_classes))

training_data = datasets.MNIST(
    root="data",
    train=True,
    download=True,
    transform=transform,
    target_transform=target_transform,
)

test_data = datasets.MNIST(
    root="data",
    train=False,
    download=True,
    transform=transform,
    target_transform=target_transform,
)

train_idx_normal = get_target_label_idx(training_data.train_labels.clone().data.cpu().numpy(), normal_classes)
train_data = Subset(training_data, train_idx_normal)

C:\Users\btdat\AppData\Local\miniconda3\envs\Deep_OOC\lib\site-packages\torchvision\datasets\mnist.py:65: UserWarning: train_labels has been renamed targets
  warnings.warn("train_labels has been renamed targets")


In [7]:
class MNIST_LeNet_AutoEncoder(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.rep_dim = 32
        self.pool = nn.MaxPool2d(2, 2)
        
        # Encoder: Same as Deep Out-of-Context (OOC) network
        self.conv1 = nn.Conv2d(1, 8, 5, bias=False, padding=2)
        self.bn1 = nn.BatchNorm2d(8, eps=1e-04, affine=False)
        self.conv2 = nn.Conv2d(8, 4, 5, bias=False, padding=2)
        self.bn2 = nn.BatchNorm2d(4, eps=1e-04, affine=False)
        self.fc1 = nn.Linear(4 * 7 * 7, self.rep_dim, bias=False)
        
        # Decoder
        self.deconv1 = nn.ConvTranspose2d(2, 4, 5, bias=False, padding=2)
        self.bn3 = nn.BatchNorm2d(4, eps=1e-04, affine=False)
        self.deconv2 = nn.ConvTranspose2d(4, 8, 5, bias=False, padding=3)
        self.bn4 = nn.BatchNorm2d(8, eps=1e-04, affine=False)
        self.deconv3 = nn.ConvTranspose2d(8, 1, 5, bias=False, padding=2)
    
    def forward(self, x):
        x = self.conv1(x)
        x = self.pool(nn.functional.leaky_relu(self.bn1(x)))
        x = self.conv2(x)
        x = self.pool(nn.functional.leaky_relu(self.bn2(x)))
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = x.view(x.size(0), int(self.rep_dim / 16), 4, 4)
        x = nn.functional.interpolate(nn.functional.leaky_relu(x), scale_factor=2)
        x = self.deconv1(x)
        x = nn.functional.interpolate(nn.functional.leaky_relu(self.bn3(x)), scale_factor=2)
        x = self.deconv2(x)
        x = nn.functional.interpolate(nn.functional.leaky_relu(self.bn4(x)), scale_factor=2)
        x = self.deconv3(x)
        x = torch.sigmoid(x)
        
        return x

In [8]:
class MNIST_LeNet_Network(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.rep_dim = 32
        self.pool = nn.MaxPool2d(2, 2)
        
        self.conv1 = nn.Conv2d(1, 8, 5, bias=False, padding=2)
        self.bn1 = nn.BatchNorm2d(8, eps=1e-04, affine=False)
        self.conv2 = nn.Conv2d(8, 4, 5, bias=False, padding=2)
        self.bn2 = nn.BatchNorm2d(4, eps=1e-04, affine=False)
        self.fc1 = nn.Linear(4 * 7 * 7, self.rep_dim, bias=False)
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.pool(nn.functional.leaky_relu(self.bn1(x)))
        x = self.conv2(x)
        x = self.pool(nn.functional.leaky_relu(self.bn2(x)))
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        return x        

In [9]:
# Configuration for Pretrain and Train
optimizer_name: str = 'adam'
lr: float = 0.001
n_epochs: int = 150
lr_milestones: tuple = ()
batch_size: int = 128
weight_decay: float = 1e-6
n_jobs_dataloader: int = 0

ae_net = MNIST_LeNet_AutoEncoder().to(device)
net = MNIST_LeNet_Network().to(device)

In [10]:
def AutoEncoder_PreTrain():
    logger = logging.getLogger()
    
    train_loader = DataLoader(train_data, batch_size, num_workers=n_jobs_dataloader)
    
    # Define loss function and optimizer
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(ae_net.parameters(), lr=lr, weight_decay=weight_decay, amsgrad=(optimizer_name == 'amsgrad'))
    schedular = optim.lr_scheduler.MultiStepLR(optimizer, milestones=lr_milestones, gamma=0.1)
    
    logger.info('Starting pretraining...')
    start_time = time.time()
    ae_net.train()
    
    # Training loop
    for epoch in range(n_epochs):
        schedular.step()
        if epoch in lr_milestones:
            logger.info('LR Scheduler: new learning rate is %g' % float(schedular.get_lr()[0]))
        loss_epoch = 0.0
        n_batches = 0
        epoch_start_time = time.time()
        for data in train_loader:
            inputs, _ = data
            inputs = inputs.to(device)
            
            # Zero the network parameters gradients
            optimizer.zero_grad()
            
            # Update network parameters via backpropagation: forward + backward + optimize
            outputs = ae_net(inputs)
            scores = torch.sum((outputs - inputs) ** 2, dim=tuple(range(1, outputs.dim())))
            loss = torch.mean(scores)
            loss.backward()
            optimizer.step()
            
            loss_epoch += loss.item()
            n_batches += 1
            
        epoch_train_time = time.time() - epoch_start_time
        logger.info('Epoch {}/{}\t Time: {:.3f}\t Loss: {:.8f}'.format(epoch + 1, n_epochs, epoch_train_time, loss_epoch / n_batches))
        
    pretrain_time = time.time() - start_time
    logger.info('Pretraining time: %.3f' % pretrain_time)
    logger.info('Finished pretraining.')
        
# AutoEncoder_PreTrain()

In [11]:
def AutoEncoder_Testing():
    from sklearn.metrics import roc_auc_score
    from sklearnex import patch_sklearn
    patch_sklearn()
    
    logger = logging.getLogger()
    
    # Get test data loader
    test_loader = DataLoader(test_data, batch_size, num_workers=n_jobs_dataloader)
    
    # Testing
    logger.info('Testing autoencoder...')
    loss_epoch = 0.0
    n_batches = 0
    start_time = time.time()
    label_score = []
    ae_net.eval()
    with torch.no_grad():
        for data in test_loader:
            inputs, labels = data
            inputs = inputs.to(device)
            outputs = ae_net(inputs)
            scores = torch.sum((outputs - inputs) ** 2, dim=tuple(range(1, outputs.dim())))
            loss = torch.mean(scores)
    
            # Save triple of (idx, label, score) in a list
            label_score += list(zip(labels.cpu().data.numpy().tolist(),
                                        scores.cpu().data.numpy().tolist()))
    
            loss_epoch += loss.item()
            n_batches += 1
    
    logger.info('Test set Loss: {:.8f}'.format(loss_epoch / n_batches))
    
    labels, scores = zip(*label_score)
    labels = np.array(labels)
    scores = np.array(scores)
    
    auc = roc_auc_score(labels, scores)
    logger.info('Test set AUC: {:.2f}%'.format(100. * auc))
    
    test_time = time.time() - start_time
    logger.info('Autoencoder testing time: %.3f' % test_time)
    logger.info('Finished testing autoencoder.')

In [12]:
# Save Pretrain model
def save_ae():
    ae_net_dict = ae_net.state_dict()
    torch.save({'ae_net_dict': ae_net_dict}, 'saved_model/ae_net.tar')

In [13]:
# Load Pretrain model
def load_ae():
    model_dict = torch.load('saved_model/ae_net.tar')
    ae_net.load_state_dict(model_dict['ae_net_dict'])

In [25]:
AutoEncoder_Testing()
load_ae()

INFO 2023-10-24 15:26:22,104 | Testing autoencoder...


Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


INFO 2023-10-24 15:26:32,811 | Test set Loss: 150.06465381
INFO 2023-10-24 15:26:32,817 | sklearn.metrics.roc_auc_score: running accelerated version on CPU
INFO 2023-10-24 15:26:33,234 | Test set AUC: 78.06%
INFO 2023-10-24 15:26:33,235 | Autoencoder testing time: 11.131
INFO 2023-10-24 15:26:33,235 | Finished testing autoencoder.


In [26]:
# Init Network Weights from Pretraining
# def init_network_weights_from_pretraining():
"""Initialize the Deep SVDD network weights from the encoder weights of the pretraining autoencoder."""

net_dict = net.state_dict()
ae_net_dict = ae_net.state_dict()

# Filter out decoder network keys
ae_net_dict = {k: v for k, v in ae_net_dict.items() if k in net_dict}
# Overwrite values in the existing state_dict
net_dict.update(ae_net_dict)
# Load the new state_dict
net.load_state_dict(net_dict)

<All keys matched successfully>

In [27]:
# Init Hypersphere
R = 0.0     # Hypersphere radius R
c = None    # Hypersphere center c

R = torch.tensor(R, device=device)
# c = torch.tensor(c, device=device)
nu: float = 0.1
assert (0 < nu) & (nu <= 1), "For hyperparameter nu, it must hold: 0 < nu <= 1."

warm_up_n_epochs = 10   # Number of training epochs for soft-boundary Deep SVDD before radius R gets updated

objective: str = 'one-class'
assert objective in ('one-class', 'soft-boundary'), "Objective must be either 'one-class' or 'soft-boundary'."

# Results
train_time = None
test_auc = None
test_time = None
test_scores = None

In [28]:
def init_center_c(train_loader: DataLoader, eps=0.1):
    """Initialize hypersphere center c as the mean from an initial forward pass on the data."""
    n_samples = 0
    c = torch.zeros(net.rep_dim, device=device)

    net.eval()
    with torch.no_grad():
        for data in train_loader:
            # get the inputs of the batch
            inputs, _ = data
            inputs = inputs.to(device)
            outputs = net(inputs)
            n_samples += outputs.shape[0]
            c += torch.sum(outputs, dim=0)

    c /= n_samples

    # If c_i is too close to 0, set to +-eps. Reason: a zero unit can be trivially matched with zero weights.
    c[(abs(c) < eps) & (c < 0)] = -eps
    c[(abs(c) < eps) & (c > 0)] = eps

    return c


def get_radius(dist: torch.Tensor, nu: float):
    """Optimally solve for radius R via the (1-nu)-quantile of distances."""
    return np.quantile(np.sqrt(dist.clone().data.cpu().numpy()), 1 - nu)

In [32]:
logger = logging.getLogger()

# Get train data loader
train_loader = DataLoader(train_data, batch_size, num_workers=n_jobs_dataloader)
    
# Set optimizer (Adam optimizer for now)
optimizer = torch.optim.Adam(ae_net.parameters(), lr=lr, weight_decay=weight_decay, amsgrad=(optimizer_name == 'amsgrad'))

# Set learning rate scheduler
scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=lr_milestones, gamma=0.1)

# Initialize hypersphere center c (if c not loaded)
if c is None:
    logger.info('Initializing center c...')
    c = init_center_c(train_loader)
    logger.info('Center c initialized.')

# Training
logger.info('Starting training...')
start_time = time.time()
net.train()
for epoch in range(n_epochs):

    scheduler.step()
    if epoch in lr_milestones:
        logger.info('  LR scheduler: new learning rate is %g' % float(scheduler.get_lr()[0]))

    loss_epoch = 0.0
    n_batches = 0
    epoch_start_time = time.time()
    for data in train_loader:
        inputs, _ = data
        inputs = inputs.to(device)

        # Zero the network parameter gradients
        optimizer.zero_grad()

        # Update network parameters via backpropagation: forward + backward + optimize
        outputs = net(inputs)
        dist = torch.sum((outputs - c) ** 2, dim=1)
        if objective == 'soft-boundary':
            scores = dist - R ** 2
            loss = R ** 2 + (1 / nu) * torch.mean(torch.max(torch.zeros_like(scores), scores))
        else:
            loss = torch.mean(dist)
        loss.backward()
        optimizer.step()

        # Update hypersphere radius R on mini-batch distances
        if (objective == 'soft-boundary') and (epoch >= warm_up_n_epochs):
            R.data = torch.tensor(get_radius(dist, nu), device=device)

        loss_epoch += loss.item()
        n_batches += 1

    # log epoch statistics
    epoch_train_time = time.time() - epoch_start_time
    logger.info('  Epoch {}/{}\t Time: {:.3f}\t Loss: {:.8f}'
                .format(epoch + 1, n_epochs, epoch_train_time, loss_epoch / n_batches))

train_time = time.time() - start_time
logger.info('Training time: %.3f' % train_time)

logger.info('Finished training.')

INFO 2023-10-24 15:27:39,684 | Initializing center c...
INFO 2023-10-24 15:27:45,995 | Center c initialized.
INFO 2023-10-24 15:27:45,995 | Starting training...


C:\Users\btdat\AppData\Local\miniconda3\envs\Deep_OOC\lib\site-packages\torch\optim\lr_scheduler.py:136: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


INFO 2023-10-24 15:27:54,723 |   Epoch 1/150	 Time: 8.728	 Loss: 7.25245537
INFO 2023-10-24 15:28:03,488 |   Epoch 2/150	 Time: 8.763	 Loss: 7.25245537
INFO 2023-10-24 15:28:09,580 |   Epoch 3/150	 Time: 6.093	 Loss: 7.25245537
INFO 2023-10-24 15:28:15,329 |   Epoch 4/150	 Time: 5.747	 Loss: 7.25245537
INFO 2023-10-24 15:28:22,274 |   Epoch 5/150	 Time: 6.945	 Loss: 7.25245537
INFO 2023-10-24 15:28:28,487 |   Epoch 6/150	 Time: 6.213	 Loss: 7.25245537
INFO 2023-10-24 15:28:36,007 |   Epoch 7/150	 Time: 7.518	 Loss: 7.25245537
INFO 2023-10-24 15:28:41,920 |   Epoch 8/150	 Time: 5.912	 Loss: 7.25245537
INFO 2023-10-24 15:28:47,318 |   Epoch 9/150	 Time: 5.397	 Loss: 7.25245537
INFO 2023-10-24 15:28:52,684 |   Epoch 10/150	 Time: 5.366	 Loss: 7.25245537
INFO 2023-10-24 15:28:58,147 |   Epoch 11/150	 Time: 5.462	 Loss: 7.25245537
INFO 2023-10-24 15:29:03,701 |   Epoch 12/150	 Time: 5.553	 Loss: 7.25245537
INFO 2023-10-24 15:29:08,937 |   Epoch 13/150	 Time: 5.235	 Loss: 7.25245537
INFO 202

In [33]:
#end train
R = float(R.cpu().data.numpy())
c = c.cpu().data.numpy().tolist()

In [30]:
def save_model():
    net_dict = net.state_dict()
    ae_net_dict = ae_net.state_dict()
    torch.save({'R': R, 'c': c, 'net_dict': net_dict, 'ae_net_dict': ae_net_dict}, 'saved_model/model.tar')

In [34]:
save_model()

In [35]:
# Load model
def load_model():
    model_dict = torch.load('saved_model/model.tar')
    
    R = model_dict['R']
    c = model_dict['c']
    net.load_state_dict(model_dict['net_dict'])
    ae_net.load_state_dict(model_dict['ae_net_dict'])

In [36]:
load_model()

In [38]:
# def test(self, dataset: BaseADDataset, net: BaseNet):
from sklearn.metrics import roc_auc_score, accuracy_score
from sklearnex import patch_sklearn
patch_sklearn()

R = torch.tensor(R, device=device)
c = torch.tensor(c, device=device)

logger = logging.getLogger()

# Get test data loader
test_loader = DataLoader(test_data, batch_size, num_workers=n_jobs_dataloader)

# Testing
logger.info('Starting testing...')
start_time = time.time()
label_score = []
net.eval()
with torch.no_grad():
    for data in test_loader:
        inputs, labels = data
        inputs = inputs.to(device)
        outputs = net(inputs)
        dist = torch.sum((outputs - c) ** 2, dim=1)
        if objective == 'soft-boundary':
            scores = dist - R ** 2
        else:
            scores = dist

        # Save triples of (idx, label, score) in a list
        label_score += list(zip(labels.cpu().data.numpy().tolist(),
                                    scores.cpu().data.numpy().tolist()))

test_time = time.time() - start_time
logger.info('Testing time: %.3f' % test_time)

test_scores = label_score

# Compute AUC
labels, scores = zip(*label_score)
labels = np.array(labels)
scores = np.array(scores)

test_auc = roc_auc_score(labels, scores)
logger.info('Test set AUC: {:.2f}%'.format(100. * test_auc))

logger.info('Finished testing.')


INFO 2023-10-24 15:55:41,775 | Starting testing...


Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)
C:\Users\btdat\AppData\Local\Temp\ipykernel_17032\4107340890.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  R = torch.tensor(R, device=device)
C:\Users\btdat\AppData\Local\Temp\ipykernel_17032\4107340890.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  c = torch.tensor(c, device=device)


INFO 2023-10-24 15:55:56,277 | Testing time: 14.499
INFO 2023-10-24 15:55:56,288 | sklearn.metrics.roc_auc_score: running accelerated version on CPU
INFO 2023-10-24 15:55:56,293 | Test set AUC: 71.13%
INFO 2023-10-24 15:55:56,295 | Finished testing.


In [60]:
test_accuracy = accuracy_score(labels, (scores > 40).astype(int))
logger.info('Test set Accuracy: {:.2f}%'.format(100. * test_accuracy))

INFO 2023-10-24 15:58:35,264 | Test set Accuracy: 9.80%


In [46]:
R, c

(tensor(0.),
 tensor([ 0.1000, -0.1000,  0.1000, -0.1000,  0.1000,  0.1000, -0.1000,  0.1000,
         -0.1000, -0.1000, -0.1000,  0.1000,  0.1000,  0.1000,  0.1000,  0.1000,
         -0.1000,  0.1000,  0.1000, -0.1000, -0.1000, -0.1000, -0.1000, -0.1000,
          0.1000,  0.1000, -0.1000, -0.1000, -0.1000, -0.1000, -0.1000,  0.1000]))